In [1]:
# Configuração do logging
import logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

In [2]:
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QComboBox, QPushButton, QFileDialog, QWidget, QVBoxLayout, QHBoxLayout, QSpacerItem, QSizePolicy, QFrame, QDesktopWidget, QScrollArea, QGridLayout, QMessageBox, QSlider
from PyQt5.QtCore import Qt, QRect, QPoint, QTimer, QFileInfo
from PyQt5.QtGui import QPainter, QPen, QImage, QPixmap, QFont
import cv2
import sys
import os
import numpy as np
from PIL import Image

## Estilo principal

In [3]:
def setup_fontes(janela):
    janela.label_fonte = QFont("Segoe UI", 20, QFont.Bold)
    janela.combo_fonte = QFont("Segoe UI", 10)
    janela.btn_fonte = QFont("Segoe UI", 12)
    
def setupStyleSheet():
    estilo = """
                QMainWindow {
                    background-color: white;
                }
                QLabel {
                    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                    font-size: 12pt;
                    color: black;
                    padding: 10px;
                    margin: 1px;
                }
                QComboBox {
                    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                    font-size: 10pt;
                    color: white;
                    background-color: #7f7f7f;
                    border-radius: 5px;
                    padding: 5px;
                    margin: 1px;
                }
                QPushButton {
                    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                    font-size: 10pt;
                    color: white;
                    background-color: #7f7f7f;
                    border-radius: 5px;
                    padding: 5px;
                }
                QPushButton:hover {
                    background-color: gray;
                }
            """
    return estilo

## Funções da Janela Principal

In [4]:
def selecionar_arq(parent, modo, tipo_arquivo):
    options = QFileDialog.Options()
    options |= QFileDialog.DontUseNativeDialog
    file_dialog = QFileDialog()
    file_dialog.setDirectory(os.path.expanduser("C:/Users/walla/documents/"))

    if tipo_arquivo == "Imagem":
        filter = "Imagens (*.png *.jpg *.jpeg)"
    elif tipo_arquivo == "Vídeo":
        filter = "Vídeos (*.mp4 *.avi)"
    else:
        filter = "Todos os Arquivos (*)"

    file_path, _ = file_dialog.getOpenFileName(
        parent, 
        "Selecionar Arquivo", 
        "", 
        filter, 
        options=options
    )

    if file_path:
        file_info = QFileInfo(file_path)
        extension = file_info.suffix().lower()
        if extension in ['png', 'jpg', 'jpeg']:
            tipo_arquivo = "Imagem"
        elif extension in ['mp4', 'avi']:
            tipo_arquivo = "Vídeo"
        else:
            print(f"Formato de arquivo não suportado: {extension}")
            return
        abrir_segunda_janela(parent, file_path, 100, 100, tipo_arquivo)

def avancar(janela):
    modo = janela.modo.currentText()
    tipo_arquivo = janela.tipo_arquivo.currentText()
    selecionar_arq(janela, modo, tipo_arquivo)

def abrir_segunda_janela(janela, file_path, x, y, tipo_arquivo):
    janela.segunda_janela = JanelaSecundaria(file_path, x, y, tipo_arquivo, janela)
    janela.segunda_janela.showMaximized()

def centralizar_tela(janela): 
    qt_rectangle = janela.frameGeometry()
    center_point = QDesktopWidget().availableGeometry().center()
    qt_rectangle.moveCenter(center_point)
    janela.move(qt_rectangle.topLeft())

## Setup widgets da Janela Principal

In [5]:
def setup_labels_tipo_arquivo(janela, fonte):
    tipo_arquivo_label = QLabel("Tipo de Arquivo", janela)
    tipo_arquivo_label.setFont(fonte)
    tipo_arquivo_label.adjustSize()
    tipo_arquivo_label.move(200, 150)
    return tipo_arquivo_label

def setup_labels_tipo_modo(janela, fonte):
    modo_label = QLabel("Selecione Modo", janela)
    modo_label.setFont(fonte)
    modo_label.setStyleSheet("padding: 10px; margin: 0px;")
    modo_label.adjustSize()
    modo_label.move(670, 150)
    return modo_label

def setup_combobox_tipo_arquivo(janela, fonte):
    tipo_arquivo = QComboBox(janela)
    tipo_arquivo.addItems(["Imagem", "Vídeo"])
    tipo_arquivo.setMinimumWidth(185)
    tipo_arquivo.setMaximumWidth(185)
    tipo_arquivo.setFont(fonte)
    tipo_arquivo.setMinimumContentsLength(10)
    tipo_arquivo.setSizeAdjustPolicy(QComboBox.AdjustToContents)
    tipo_arquivo.move(200, 200)
    return tipo_arquivo
    
def setup_combobox_modo(janela, fonte):
    modo = QComboBox(janela)
    modo.addItems(["Independente", "Cascata"])
    modo.setMinimumWidth(185)
    modo.setMaximumWidth(185)
    modo.setFont(fonte)
    modo.setMinimumContentsLength(10)
    modo.setSizeAdjustPolicy(QComboBox.AdjustToContents)
    modo.move(670, 200)
    return modo

def setup_btn_avancar(janela, fonte):
    btn_avancar = QPushButton("Avançar", janela)
    btn_avancar.setFont(fonte)
    btn_avancar.resize(120, 40)
    btn_avancar.move(200, 600)
    btn_avancar.clicked.connect(lambda: avancar(janela))
    return btn_avancar

def setup_btn_sair(janela, fonte):
    btn_sair = QPushButton("Sair", janela)
    btn_sair.setFont(fonte)
    btn_sair.setStyleSheet("padding: 10px 45px;")
    btn_sair.resize(120, 40)
    btn_sair.move(670, 600)
    btn_sair.clicked.connect(janela.close)
    return btn_sair

## Janela Principal

In [6]:
class JanelaPrincipal(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Manipulação de Imagens e Vídeos")
        self.setGeometry(100, 100, 1000, 700)
        self._setup_ui()

    def _setup_ui(self):
        self.setStyleSheet(setupStyleSheet())
        centralizar_tela(self)
        setup_fontes(self)
        self.tipo_arquivo_label = setup_labels_tipo_arquivo(self, self.label_fonte)
        self.modo_label = setup_labels_tipo_modo(self, self.label_fonte)
        self.modo = setup_combobox_modo(self, self.combo_fonte)
        self.tipo_arquivo = setup_combobox_tipo_arquivo(self, self.combo_fonte)
        self.btn_avancar = setup_btn_avancar(self, self.btn_fonte)
        self.btn_sair = setup_btn_sair(self, self.btn_fonte)

## Widgets da janela secundária

In [7]:
def setup_label_filtro_j2(janela):
    filtro_label = QLabel("Aplicar Filtro", janela)
    filtro_label.setFont(QFont("Segoe UI", 12))
    filtro_label.setStyleSheet("background-color: transparent;")
    filtro_label.move(30, 130)  # Coordenadas x, y
    filtro_label.adjustSize()  # Ajustar tamanho ao texto
    return filtro_label

def setup_label_filtro_combo_j2(janela):
    filtro_combo = QComboBox(janela)
    filtro_combo.addItems(["blur", "sharpen", "emboss", "laplacian", "canny", "sobel", "cinza", "binario", "cores"])
    filtro_combo.setMinimumWidth(300)
    filtro_combo.setMaximumWidth(300)
    filtro_combo.move(30, 210)  # Coordenadas x, y
    filtro_combo.currentIndexChanged.connect(janela.aplicar_filtro_selecionado)
    return filtro_combo

def setup_btn_webcam_j2(janela):
    btn_webcam = QPushButton("Abrir Webcam", janela)
    btn_webcam.setFixedSize(140, 40)
    btn_webcam.move(600, 130)  # Coordenadas x, y
    btn_webcam.setVisible(False)
    btn_webcam.clicked.connect(janela.abrir_webcam)
    return btn_webcam

def setup_btn_play_j2(janela): 
    btn_play = QPushButton("Play", janela)
    btn_play.setFixedSize(140, 40)
    btn_play.move(780, 130)  # Coordenadas x, y
    btn_play.setVisible(False)
    btn_play.clicked.connect(janela.play_video)
    return btn_play

def setup_btn_pause_j2(janela):
    btn_pause = QPushButton("Pause", janela)
    btn_pause.setFixedSize(140, 40)
    btn_pause.move(960, 130)  # Coordenadas x, y
    btn_pause.setVisible(False)
    btn_pause.clicked.connect(janela.pause_video)
    return btn_pause

def setup_combobox_velocidade(janela):
    combo_velocidade = QComboBox(janela)
    combo_velocidade.addItems(["0.25x", "0.5x", "1.0x", "2.0x", "3.0x"])
    #combo_velocidade.currentIndexChanged.connect(janela.ajustar_velocidade_video)
    combo_velocidade.setCurrentText("1.0x")  # Velocidade padrão
    combo_velocidade.setFixedSize(140, 40)
    combo_velocidade.move(1458, 168)  # Coordenadas x, y
    return combo_velocidade

def setup_btn_avancar_j2(janela):
    btn_avancar = QPushButton("Acelerar", janela)
    btn_avancar.setFixedSize(140, 40)
    btn_avancar.move(1458, 130)  # Coordenadas x, y
    btn_avancar.clicked.connect(janela.ajustar_velocidade_video)
    btn_avancar.setVisible(False)
    return btn_avancar


def setup_btn_inverter_j2(janela):
    btn_inverter = QPushButton("Inverter", janela)
    btn_inverter.setFixedSize(140, 40)
    btn_inverter.move(1300, 130)  # Coordenadas x, y
    btn_inverter.setVisible(False)
    btn_inverter.clicked.connect(janela.inverter_video)
    return btn_inverter

def setup_btn_stop_j2(janela):
    btn_stop = QPushButton("Parar", janela)
    btn_stop.setFixedSize(140, 40)
    btn_stop.move(1130, 130)  # Coordenadas x, y
    btn_stop.setVisible(False)
    btn_stop.clicked.connect(janela.stop_all)
    return btn_stop

def setup_btn_zoom_mais_j2(janela):
    btn_zoom_mais = QPushButton("Zoom+", janela)
    btn_zoom_mais.setFixedSize(140, 40)
    btn_zoom_mais.move(1260, 910)
    btn_zoom_mais.clicked.connect(janela.zoom_in)
    return btn_zoom_mais

def setup_btn_zoom_menos_j2(janela):
    btn_zoom_menos = QPushButton("Zoom-", janela)
    btn_zoom_menos.setFixedSize(140, 40)
    btn_zoom_menos.move(1458, 910)
    btn_zoom_menos.clicked.connect(janela.zoom_out)
    return btn_zoom_menos
    
def setup_btn_salvar_j2(janela):
    btn_salvar = QPushButton("Salvar", janela)
    btn_salvar.setFixedSize(140, 40)
    btn_salvar.move(1050, 910)
    btn_salvar.clicked.connect(janela.salvar_recorte)
    return btn_salvar

def setup_btn_desfazer_j2(janela):
    btn_desfazer = QPushButton("Desfazer Seleção", janela)
    btn_desfazer.setFixedSize(140, 40)
    btn_desfazer.move(600, 910)
    btn_desfazer.clicked.connect(janela.aplicar_tamanho_original)
    return btn_desfazer

def setup_btn_aplicar_filtro_j2(janela):
    btn_aplicar_filtro = QPushButton("Aplicar Filtro", janela)
    btn_aplicar_filtro.setFixedSize(140, 40)
    btn_aplicar_filtro.move(830, 910)
    btn_aplicar_filtro.clicked.connect(janela.aplicar_filtro_selecionado)
    return btn_aplicar_filtro

def setup_btn_voltar_tela_j2(janela):
    btn_voltar_tela = QPushButton("Voltar", janela)
    btn_voltar_tela.setFixedSize(140, 40)
    btn_voltar_tela.move(35, 910)  # Coordenadas x, y
    btn_voltar_tela.clicked.connect(lambda: voltar(janela))
    return btn_voltar_tela

def setup_btn_sair_j2(janela):
    btn_sair = QPushButton("Sair", janela)
    btn_sair.setFixedSize(140, 40)
    btn_sair.move(300, 910)  # Coordenadas x, y
    btn_sair.clicked.connect(lambda: sair(janela))
    return btn_sair

def setup_scroll_area_j2(janela):
    scroll_area = QScrollArea(janela)
    scroll_area.setFixedSize(1000, 600)
    scroll_area.setWidgetResizable(True)
    scroll_area.setAlignment(Qt.AlignCenter)
    scroll_area.move(600, 200)  # Coordenadas x, y
    return scroll_area

def setup_image_label_j2(janela):
    image_label = QLabel(janela)
    image_label.setAlignment(Qt.AlignCenter)
    janela.scroll_area.setWidget(image_label)
    return image_label

def setup_slider_j2(janela):
    slider = QSlider(Qt.Horizontal, janela)
    janela.time_label = QLabel(janela)
    slider.setGeometry(600, 830, 1000, 30)  # Definindo coordenadas x e y
    slider.setRange(0, 100)
    slider.sliderMoved.connect(janela.set_video_position)
    return slider
    
def setup_time_label_j2(janela):
    time_label = QLabel(janela)
    time_label.setGeometry(410, 827, 180, 35)  # Definindo coordenadas x e y
    time_label.setText("00:00:00 / 00:00:00")
    time_label.setStyleSheet("font-family: 'Segoe UI'; font-size: 10pt;")
    return time_label

## Funções da Janela Secundária

In [8]:
def voltar(janela): 
    janela.close() 
    janela.janela_principal.show()

def sair(janela): 
    reply = QMessageBox.question(janela, 'Confirmação', 'Tem certeza de que deseja sair?', QMessageBox.Sim | QMessageBox.Não, QMessageBox.No) 
    if reply == QMessageBox.Yes: 
        janela.janela_principal.close() 
        janela.close()


## Janela Secundária

In [9]:
class JanelaSecundaria(QMainWindow):
    def __init__(self, file_path, x, y, tipo_arquivo, parent):
        super().__init__(parent)
        self.setWindowTitle("Manipulação de Imagens e Vídeos")
        self.setGeometry(x, y, 1200, 800)
        self._setup_ui()
        self.file_path = file_path
        self.tipo_arquivo = tipo_arquivo
        self.janela_principal = parent
        self.original_image = None
        self.scaled_image = None  # Nova variável para armazenar a imagem escalada
        self.filtered_image = None # Nova variável para armazenar a imagem filtrada
        self.display_image = None # Variável para armazenar a imagem exibida (com zoom aplicado)
        self.zoom_level = 1.0 # Fator de zoom inicial
        self.initial_zoom_level = self.zoom_level
        self.zoom_in_count = 0 # Contador de cliques no botão Zoom+
        self.start_point = QPoint()# Ponto inicial da seleção de recorte
        self.end_point = QPoint() # Ponto final da seleção de recorte
        self.is_selecting = False # Flag para indicar se o recorte está sendo selecionado
        self.timer = QTimer()
        self.capturando_webcam = False
        self.showMaximized()
        self.carregar_arquivo(file_path)


    def _setup_ui(self):
        central_widget = QWidget(self)
        self.setCentralWidget(central_widget)
        self.setStyleSheet(setupStyleSheet())
        self.filtro_label = setup_label_filtro_j2(self)
        self.filtro_combo = setup_label_filtro_combo_j2(self)
        self.btn_webcam = setup_btn_webcam_j2(self)
        self.btn_play = setup_btn_play_j2(self)
        self.btn_pause = setup_btn_pause_j2(self)
        self.btn_avancar = setup_btn_avancar_j2(self)
        self.btn_inverter = setup_btn_inverter_j2(self)
        self.btn_stop = setup_btn_stop_j2(self)
        self.btn_zoom_mais = setup_btn_zoom_mais_j2(self)
        self.btn_zoom_menos = setup_btn_zoom_menos_j2(self)
        self.btn_salvar = setup_btn_salvar_j2(self)
        self.btn_desfazer = setup_btn_desfazer_j2(self)
        self.btn_aplicar_filtro = setup_btn_aplicar_filtro_j2(self)
        self.btn_voltar_tela = setup_btn_voltar_tela_j2(self)
        self.btn_sair = setup_btn_sair_j2(self)
        self.scroll_area = setup_scroll_area_j2(self)
        self.image_label = setup_image_label_j2(self)
        self.slider = setup_slider_j2(self)
        self.time_label = setup_time_label_j2(self)
        self.combo_velocidade = setup_combobox_velocidade(self)
        
        self.slider.setVisible(False) 
        self.time_label.setVisible(False)
        self.combo_velocidade.setVisible(False)
        self.btn_avancar.setVisible(False)


    def ajustar_velocidade_video(self):
        velocidade_text = self.combo_velocidade.currentText()
        velocidade = float(velocidade_text[:-1])  # Remove o "x" e converte para float
        self.timer.setInterval(int(30 / velocidade))
        self.combo_velocidade.currentIndexChanged.connect(self.ajustar_velocidade_video)
        logging.debug(f"Velocidade do vídeo ajustada para: {velocidade_text}")

            
    def atualizar_imagem(self):
        # Usar a imagem filtrada se existir, caso contrário usar a original
        imagem_para_exibir = self.filtered_image if self.filtered_image is not None else self.original_image
    
        if imagem_para_exibir:
            # Calcular o novo tamanho com base no zoom
            largura, altura = imagem_para_exibir.size
            nova_largura = int(largura * self.zoom_level)
            nova_altura = int(altura * self.zoom_level)
        
            # Redimensionar a imagem mantendo o filtro
            imagem_zoom = imagem_para_exibir.resize((nova_largura, nova_altura), Image.LANCZOS)
        
            # Converter a imagem PIL para QPixmap
            pixmap = self.pil_imagem_qpixmap(imagem_zoom)
        
            # Definir a imagem redimensionada no QLabel
            self.image_label.setPixmap(pixmap)
            self.image_label.adjustSize()
            self.scroll_area.setWidget(self.image_label)
        
            logging.debug(f"Imagem atualizada com fator de zoom: {self.zoom_level}")
        else:
            logging.error("Nenhuma imagem disponível para exibição!")
            
    def zoom_in(self):
        if self.zoom_in_count == 0:
            # Na primeira vez, definir o zoom inicial corretamente
            self.zoom_level = 1.1
        else:
            # Nos próximos cliques, aumentar progressivamente
            self.zoom_level *= 1.1
        self.zoom_in_count += 1
        logging.debug(f"Zoom in: level {self.zoom_level}, count {self.zoom_in_count}")
        self.atualizar_imagem()

    def zoom_out(self):
        if self.zoom_in_count > 1:
            self.zoom_level /= 1.1
            self.zoom_in_count -= 1
            logging.debug(f"Zoom out: level {self.zoom_level}, count {self.zoom_in_count}")
            self.atualizar_imagem()
        else:
            self.aplicar_tamanho_original()

    def carregar_arquivo(self, file_path):
        logging.debug(f"Carregando arquivo do caminho: {file_path}")
        if self.tipo_arquivo == "Imagem":
            try:
                self.original_image = Image.open(file_path)
                logging.debug("Imagem carregada com sucesso.")
                self.carregar_imagem()
                # Tornar os botões de vídeo e barra de progresso invisíveis
                self._set_video_controls_visible(False)
                self.slider.setVisible(False)
                self.time_label.setVisible(False)
            except Exception as e:
                logging.error(f"Falha ao carregar a imagem: {e}")
        elif self.tipo_arquivo == "Vídeo":
            try:
                self.lista = []
                self.video_capture = cv2.VideoCapture(file_path)
                if not self.video_capture.isOpened():
                    raise ValueError("Erro ao abrir o vídeo.")
                logging.debug("Vídeo carregado com sucesso.")
                self.timer.timeout.connect(self.atualizar_quadro)
                self.processar_video()
                # Tornar os botões de vídeo e barra de progresso visíveis
                self._set_video_controls_visible(True)
                self.slider.setVisible(True)
                self.time_label.setVisible(True)
            except Exception as e:
                logging.error(f"Falha ao carregar o vídeo: {e}")

    def _set_video_controls_visible(self, visible):
        self.btn_webcam.setVisible(visible)
        self.btn_play.setVisible(visible)
        self.btn_pause.setVisible(visible)
        self.btn_avancar.setVisible(visible)
        self.btn_avancar.setVisible(visible)
        self.btn_stop.setVisible(visible)
        self.btn_inverter.setVisible(visible)
        self.combo_velocidade.setVisible(visible)

    """def abrir_webcam(self):
        cap = cv2.VideoCapture(0)  # Abre a primeira webcam conectada
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            self.exibir_frame(frame)
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()"""

    def abrir_webcam(self):
        self.capturando_webcam = True
        self.cap = cv2.VideoCapture(1)  # Abre a primeira webcam conectada
        self.timer.timeout.connect(self.atualizar_frame_webcam)
        self.timer.start(30)  # Atualiza a cada 30ms

    def atualizar_frame_webcam(self):
        if self.capturando_webcam:
            ret, frame = self.cap.read()
            if not ret:
                return
            self.exibir_frame(frame)
        else:
            self.cap.release()
            self.timer.stop()
            cv2.destroyAllWindows()

    def parar_webcam(self):
        self.capturando_webcam = False
        self.cap.release()
        self.timer.stop()
        cv2.destroyAllWindows()
    
    def stop_all(self):
        if self.capturando_webcam:
            self.parar_webcam()
        else:
            self.stop_video()
    
    def stop_video(self):
        self.timer.stop()
        self.video_capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
        self.atualizar_quadro()
        logging.debug("Reprodução do vídeo parada.")
    
    # Resto do código da JanelaSecundaria continua...


    def reverso(self, video):
        self.lista = []
        cap = cv2.VideoCapture(video)
        while True:
            rval, frame = cap.read()
            if not rval:
                break
            self.lista.append(frame)
        self.lista.reverse()
        for frame in self.lista:
            self.exibir_frame(frame)
            key = cv2.waitKey(100) & 0xFF
            if key == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()

    def inverter_video(self):
        if self.video_capture.isOpened():
            self.reverso(self.file_path)

    def exibir_frame(self, frame):
        rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pixmap = self.opencv_imagem_qpixmap(rgb_image)
        self.image_label.setPixmap(pixmap.scaled(1000, 500, Qt.KeepAspectRatio, Qt.SmoothTransformation))
    
    def pil_imagem_qpixmap(self, image):
        """Converte uma imagem PIL para QPixmap"""
        image = image.convert("RGBA")
        date = image.tobytes("raw", "RGBA")
        qimage = QImage(date, image.width, image.height, QImage.Format_RGBA8888)
        pixmap = QPixmap.fromImage(qimage)
        return pixmap

    def carregar_imagem(self):
        if self.original_image:
            pixmap = self.pil_imagem_qpixmap(self.original_image)
            self.image_label.setPixmap(pixmap.scaled(self.image_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
            self.image_label.adjustSize()
            self.scroll_area.setWidget(self.image_label)
            logging.debug("Imagem exibida no QLabel.")
        else:
            logging.error("original_image não está definida!")


    def aplicar_filtro_completo(self, image, filter_type):
        cv_image = np.array(image)

        if filter_type == "blur":
            logging.debug("Aplicando filtro: blur")
            filtered_image = cv2.GaussianBlur(cv_image, (15, 15), 0)
        elif filter_type == "sharpen":
            logging.debug("Aplicando filtro: sharpen")
            kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
            filtered_image = cv2.filter2D(cv_image, -1, kernel)
        elif filter_type == "emboss":
            logging.debug("Aplicando filtro: emboss")
            kernel = np.array([[0, -1, -1], [1, 0, -1], [1, 1, 0]])
            filtered_image = cv2.filter2D(cv_image, -1, kernel)
        elif filter_type == "laplacian":
            logging.debug("Aplicando filtro: laplacian")
            filtered_image = cv2.Laplacian(cv_image, cv2.CV_64F)
            filtered_image = cv2.convertScaleAbs(filtered_image)
        elif filter_type == "canny":
            logging.debug("Aplicando filtro: canny")
            filtered_image = cv2.Canny(cv_image, 100, 200)
        elif filter_type == "sobel":
            logging.debug("Aplicando filtro: sobel")
            filtered_image = cv2.Sobel(cv_image, cv2.CV_64F, 1, 0, ksize=5)
            filtered_image = cv2.convertScaleAbs(filtered_image)
        elif filter_type == "median_blur":
            logging.debug("Aplicando filtro: median_blur")
            filtered_image = cv2.medianBlur(cv_image, 5)
        elif filter_type == "bilateral_filter":
            logging.debug("Aplicando filtro: bilateral_filter")
            filtered_image = cv2.bilateralFilter(cv_image, 9, 75, 75)
        elif filter_type == "scharr":
            logging.debug("Aplicando filtro: scharr")
            filtered_image = cv2.Scharr(cv_image, cv2.CV_64F, 1, 0)
        elif filter_type == "prewitt":
            logging.debug("Aplicando filtro: prewitt")
            kernelx = np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]])
            kernely = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]])
            filtered_image_x = cv2.filter2D(cv_image, -1, kernelx)
            filtered_image_y = cv2.filter2D(cv_image, -1, kernely)
            filtered_image = cv2.addWeighted(filtered_image_x, 0.5, filtered_image_y, 0.5, 0)
        elif filter_type == "binario":
            logging.debug("Aplicando filtro: binario")
            _, filtered_image = cv2.threshold(cv_image, 127, 255, cv2.THRESH_BINARY)
        elif filter_type == "cinza":
            logging.debug("Aplicando filtro: cinza")
            filtered_image = cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY)
        elif filter_type == "cores":
            logging.debug("Aplicando filtro: cores")
            filtered_image = cv2.cvtColor(cv_image, cv2.COLOR_BGR2HSV)
        else:
            filtered_image = cv_image

        logging.debug(f"Filtro {filter_type} aplicado.")
        return Image.fromarray(filtered_image)

    def aplicar_filtro_selecionado(self):
        filter_type = self.filtro_combo.currentText()
        # Aplicar o filtro na imagem original
        self.filtered_image = self.aplicar_filtro_completo(self.original_image, filter_type)
        # Resetar o zoom para o nível inicial
        self.zoom_level = 1.0
        self.zoom_in_count = 0
        # Atualizar a imagem com o filtro aplicado
        self.atualizar_imagem_filtrada()


    def atualizar_imagem_filtrada(self):
        if self.filtered_image:
            pixmap = self.pil_imagem_qpixmap(self.filtered_image)
            self.image_label.setPixmap(pixmap.scaled(self.image_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
            self.image_label.adjustSize()
            self.scroll_area.setWidget(self.image_label)
            logging.debug("Imagem filtrada atualizada.")

    def mostrar_mensagem_fim_video(self):
        frame_count = len(os.listdir(self.output_dir))
        mensagem = QMessageBox(self)
        mensagem.setWindowTitle("Vídeo Finalizado")
        mensagem.setText(f"O vídeo finalizou! Foram gerados {frame_count} frames.\nVocê deseja salvar em uma pasta local?")
        mensagem.setStandardButtons(QMessageBox.Yes | QMessageBox.No)
        resposta = mensagem.exec_()

        if resposta == QMessageBox.Yes:
            self.salvar_frames_em_pasta()
        else:
            # Limpar frames temporários se o usuário não quiser salvar
            self.limpar_frames_temporarios()

    def atualizar_quadro(self):
        ret, frame = self.video_capture.read()
        if ret:
            imagem = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Salve o frame como arquivo de imagem
            self.salvar_frame(imagem)

            pixmap = self.opencv_imagem_qpixmap(imagem)
            self.image_label.setPixmap(pixmap.scaled(self.scroll_area.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
            self.image_label.adjustSize()
            self.scroll_area.setWidget(self.image_label)
            logging.debug("Quadro de vídeo atualizado.")
        else:
            self.timer.stop()
            self.mostrar_mensagem_fim_video()
            logging.debug("Fim do vídeo ou erro ao ler o quadro.")

    def format_time(self, seconds):
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        seconds = int(seconds % 60)
        return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

    def processar_video(self):
        self.timer.start(30)
        logging.debug("Iniciando processamento do vídeo.")

    def set_video_position(self, position):
        total_frames = int(self.video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
        new_frame = int((position / 100) * total_frames)
        self.video_capture.set(cv2.CAP_PROP_POS_FRAMES, new_frame)
        self.atualizar_quadro()

    
    def opencv_imagem_qpixmap(self, image):
        altura, largura, canal = image.shape
        bytes_por_linha = canal * largura
        qimage = QImage(image.data, largura, altura, bytes_por_linha, QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(qimage)
        return pixmap

    def aplicar_tamanho_original(self):
        if self.original_image is not None:
            try:
                # Converta a imagem PIL para um array numpy
                original_img_np = cv2.cvtColor(np.array(self.original_image), cv2.COLOR_RGB2BGR)
                pixmap = self.pil_imagem_qpixmap(self.original_image)
                self.image_label.setPixmap(pixmap.scaled(self.scroll_area.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
                self.image_label.resize(self.scroll_area.size())
                self.scroll_area.setWidget(self.image_label)
                self.scroll_area.setWidgetResizable(True)
                print("Imagem redefinida para o tamanho original.")
            except Exception as e:
                print(f"Erro ao redefinir para o tamanho original: {e}")
        else:
            print("Nenhuma imagem carregada para redefinir.")

    def play_video(self):
        if not self.timer.isActive():
            self.timer.start(30)
            logging.debug("Reprodução do vídeo iniciada.")

    def pause_video(self):
        if self.timer.isActive():
            self.timer.stop()
            logging.debug("Reprodução do vídeo pausada.")

    def stop_video(self):
        self.timer.stop()
        self.video_capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
        self.atualizar_quadro()
        logging.debug("Reprodução do vídeo parada.")


    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            # Verifica se o clique está dentro do QScrollArea
            if self.scroll_area.geometry().contains(event.pos()):
                self.start_point = self.mapFromGlobal(event.globalPos()) - self.scroll_area.pos()
                self.end_point = self.start_point
                self.is_selecting = True
                logging.debug(f"Seleção iniciada no ponto: {self.start_point}")
            else:
                self.is_selecting = False

    def mouseMoveEvent(self, event):
        if self.is_selecting:
            self.end_point = self.mapFromGlobal(event.globalPos()) - self.scroll_area.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton and self.is_selecting:
            self.end_point = self.mapFromGlobal(event.globalPos()) - self.scroll_area.pos()
            self.is_selecting = False
            # Verifique se o retângulo de seleção é válido
            if self.start_point != self.end_point and self.scroll_area.geometry().contains(self.mapToGlobal(self.end_point)):
                self.recortar_imagem()
            logging.debug(f"Seleção finalizada no ponto: {self.end_point}")

    def recortar_imagem(self):
        # Determinar qual imagem usar para recorte
        imagem_para_recorte = self.filtered_image if self.filtered_image is not None else self.original_image

        if imagem_para_recorte is not None and not self.start_point.isNull() and not self.end_point.isNull():
            try:
                # Calcula as coordenadas relativas na QLabel
                x1_rel = min(self.start_point.x(), self.end_point.x()) - self.scroll_area.widget().x()
                y1_rel = min(self.start_point.y(), self.end_point.y()) - self.scroll_area.widget().y()
                x2_rel = max(self.start_point.x(), self.end_point.x()) - self.scroll_area.widget().x()
                y2_rel = max(self.start_point.y(), self.end_point.y()) - self.scroll_area.widget().y()

                # Ajustar as proporções para corresponder à imagem original
                escala_x = imagem_para_recorte.width / self.image_label.width()
                escala_y = imagem_para_recorte.height / self.image_label.height()

                # Converte as coordenadas relativas para coordenadas na imagem
                x1_img = int(x1_rel * escala_x)
                y1_img = int(y1_rel * escala_y)
                x2_img = int(x2_rel * escala_x)
                y2_img = int(y2_rel * escala_y)

                # Verifica se a área de recorte é válida
                if x1_img < x2_img and y1_img < y2_img:
                    # Garantir que o recorte esteja dentro dos limites da imagem
                    x1_img = max(0, x1_img)
                    y1_img = max(0, y1_img)
                    x2_img = min(imagem_para_recorte.width, x2_img)
                    y2_img = min(imagem_para_recorte.height, y2_img)

                    self.recorte = imagem_para_recorte.crop((x1_img, y1_img, x2_img, y2_img))
                
                    # Converter para RGB se necessário para garantir exibição correta
                    if self.recorte.mode != 'RGB':
                        self.recorte = self.recorte.convert('RGB')
                
                    pixmap = self.pil_imagem_qpixmap(self.recorte)
                    self.image_label.setPixmap(pixmap)
                    self.image_label.adjustSize()
                    self.scroll_area.setWidget(self.image_label)
                    logging.debug(f"Imagem recortada: {x1_img}, {y1_img}, {x2_img}, {y2_img}")
            except Exception as e:
                logging.error(f"Erro ao recortar imagem: {e}")

    def paintEvent(self, event):
        super().paintEvent(event)
        if self.is_selecting:
            painter = QPainter(self)
            pen = QPen(Qt.red, 2, Qt.SolidLine)
            painter.setPen(pen)
            rect = QRect(self.start_point, self.end_point)
            painter.drawRect(rect)
            if self.scroll_area.geometry().contains(self.mapToGlobal(self.start_point)) and self.scroll_area.geometry().contains(self.mapToGlobal(self.end_point)):
                painter.drawRect(rect)

    def salvar_frame(self, frame):
        # Diretório onde os frames serão salvos temporariamente
        self.output_dir = "frames_temp"
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)

        # Nome do arquivo de frame
        frame_number = int(self.video_capture.get(cv2.CAP_PROP_POS_FRAMES))
        frame_filename = os.path.join(self.output_dir, f"frame_{frame_number:04d}.png")

        # Salvar o frame
        cv2.imwrite(frame_filename, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
        logging.debug(f"Frame salvo: {frame_filename}")

    def salvar_frames_em_pasta(self):
        pasta_destino = QFileDialog.getExistingDirectory(self, "Selecione a Pasta de Destino", "C:/")
        if pasta_destino:
            for filename in os.listdir(self.output_dir):
                src_path = os.path.join(self.output_dir, filename)
                dst_path = os.path.join(pasta_destino, filename)
                os.rename(src_path, dst_path)
            logging.debug(f"Frames salvos na pasta: {pasta_destino}")
        self.limpar_frames_temporarios()

    def limpar_frames_temporarios(self):
        for filename in os.listdir(self.output_dir):
            os.remove(os.path.join(self.output_dir, filename))
        os.rmdir(self.output_dir)
        logging.debug("Frames temporários limpos.")


    def salvar_recorte(self):
        if hasattr(self, 'recorte'):
            file_path, _ = QFileDialog.getSaveFileName(self, "Salvar Recorte", "C:/", "Imagens (*.png *.jpg *.jpeg)")
            if file_path:
                self.recorte.save(file_path)
                logging.debug(f"Imagem recortada salva em: {file_path}")
        else:
            logging.error("Nenhum recorte disponível para salvar.")

In [10]:
if __name__ == "__main__":
    app = QApplication(sys.argv)
    janela_principal = JanelaPrincipal()
    janela_principal.show()
    app.exec_()

2024-12-23 15:35:30,538 - DEBUG - Carregando arquivo do caminho: C:/Users/user/Desktop/Estudos/teti/trabalhos/manipulacao_video_img/imagens/objetos.jpg
2024-12-23 15:35:30,588 - DEBUG - tag: ImageWidth (256) - type: short (3) - value: b'\x12 '
2024-12-23 15:35:30,589 - DEBUG - tag: ImageLength (257) - type: short (3) - value: b'\n2'
2024-12-23 15:35:30,590 - DEBUG - tag: Make (271) - type: string (2) Tag Location: 46 - Data Location: 158 - value: b'motorola\x00'
2024-12-23 15:35:30,591 - DEBUG - tag: Model (272) - type: string (2) Tag Location: 58 - Data Location: 168 - value: b'motorola edge 30 neo\x00'
2024-12-23 15:35:30,593 - DEBUG - tag: Orientation (274) - type: short (3) - value: b'\x00\x06'
2024-12-23 15:35:30,594 - DEBUG - tag: XResolution (282) - type: rational (5) Tag Location: 82 - Data Location: 190 - value: b'\x00\x00\x00H\x00\x00\x00\x01'
2024-12-23 15:35:30,596 - DEBUG - tag: YResolution (283) - type: rational (5) Tag Location: 94 - Data Location: 198 - value: b'\x00\x0

Imagem redefinida para o tamanho original.


AttributeError: type object 'QMessageBox' has no attribute 'Sim'

In [11]:
JanelaPrincipal()